## Preperation
Install packages, change working directory, defining functions that will write result to output. This part is the same as BART, so refer back to BART.

In [1]:
!pip install datasets
# !pip install accelerate -U
from datasets import load_dataset, load_from_disk
import torch
import os
import numpy as np
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import get_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import functional as F
import json
from torch.nn import CrossEntropyLoss
# Mount to google drive either click it or add a block cell
# Change it to your google drive path where this notebook located.
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/Baselines/Seq2Seq'
os.chdir(drive_path)

RECORD_HYPERPARAMETERS = 'record_hyperparameters'
RECORD_TRAIN_LOSS_AND_ACCURACY = 'record_train_loss_and_accuracy'
RECORD_TEST_LOSS_AND_ACCURACY = 'record_test_loss_and_accuracy'
RECORD_MODEL = 'record_model'

def write_results(output_dir, result_type, results, **kwargs):
    # write the hyper parameter under output_dir
    if result_type == RECORD_HYPERPARAMETERS:
        hyper_parameters_file = os.path.join(output_dir, 'hyper_parameters.json')
        with open(hyper_parameters_file, 'w') as f:
            json.dump(hyper_parameters, f)
            return
    # write the train and validate results under output_dir
    if result_type == RECORD_TRAIN_LOSS_AND_ACCURACY:
        json_file = os.path.join(output_dir,f'train_metrics.json')
        if os.path.exists(json_file):
            with open(json_file, 'r') as file:
                data = json.load(file)
        else:
            # Initialize data as an empty dictionary or appropriate structure
            data = []

        # Append the results to the existing data
        data.append(results)
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)
            return

    # write the validate and test results under output_dir
    if result_type == RECORD_TEST_LOSS_AND_ACCURACY:
        json_file = os.path.join(output_dir, f'validate_and_test_metrics.json')
        if os.path.exists(json_file):
            with open(json_file, 'r') as file:
                data = json.load(file)
        else:
            # Initialize data as an empty dictionary or appropriate structure
            data = []

        # Append the results to the existing data
        data.append(results)
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)
            return

    # store the model under output_dir
    if result_type == RECORD_MODEL:
        results.save_pretrained(output_dir)
        return


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

## Training Functions
Defining training functions: train batch, train epoch, train model, evaluate model. The definition will be different from BART, because the model takes decoder_token_ids as input (in addtion to the regular input_token_ids and attention_masks).  
What is decoder_token_ids? [Link](https://huggingface.co/docs/transformers/en/glossary#decoder-input-ids)

So basically we need to pass in labels, so that when the decoder are generating, it will look at the gold standards' previous word and generate next word, this is called teacher forcing, else, the model will generate the next word based on its previous generated words, so it might diverge from truth more and more as time goes on. So passing the labels is a good approach.

In [2]:
EVAL_SUBSAMPLE_SIZE = 100
VAL_SET_SIZE = 1000
TEST_SET_SIZE = 1000

EVAL_PER_STEP = 100

def calculate_accuracy(logits, labels, tokenizer):
    predictions = torch.argmax(logits, dim=-1)
    # flatten the predictions and labels
    predictions = predictions.view(-1)
    labels = labels.view(-1)
    # calculate correct predictions
    correct_labels, total_labels = 0, 0
    for i in range(len(predictions)):
        if predictions[i] == tokenizer.pad_token_id and labels[i] == tokenizer.pad_token_id:
            continue
        if predictions[i] == labels[i]:
            correct_labels += 1
        total_labels += 1
    accuracy = correct_labels / total_labels
    return accuracy

def customize_loss_and_accuracy(outputs, target, tokenizer):
    # make the input and target the correct size (input is (batch* seq_len, dictionary_size), output is (batch*seq_len))
    loss = F.cross_entropy(input=outputs.logits.view(-1, outputs.logits.size(-1)), target=target.view(-1))
    accuracy = calculate_accuracy(logits=outputs.logits, labels=target, tokenizer=tokenizer)
    return loss, accuracy

def train_batch(model, tokenizer, epoch, step, batch, device, optimizer, scheduler, epoch_dir):
    # set model to train mode
    model.train()

    # put everything on the right device
    batch =  {k: v.to(device) for k, v in batch.items()}
    batch_size = batch['labels'].shape[0]

    # clear gradients, same old as usual
    optimizer.zero_grad()

    # forward pass, T5 forced us to pass in labels, this is good for "teacher forcing" according to ChatGPT.
    outputs = model(**batch)

    # outputs.loss might be problematic because of the NllLossBackward0 without softmax, should use nn.CrossEntropy
    loss, accuracy = customize_loss_and_accuracy(outputs, target=batch['labels'], tokenizer=tokenizer)

    # back propagation
    loss.backward()
    optimizer.step()

    # scheduler adjust lr
    scheduler.step()

    # record the train loss and accuracy
    record = {"evaluate_set": 'train', "epoch":epoch, "batch":step,
              "avg_loss":loss.item()/batch_size, "accuracy":accuracy, 'subsample_size':"None"}
    print(record)
    # WRITE: save the result for this epoch
    write_results(epoch_dir, result_type=RECORD_TRAIN_LOSS_AND_ACCURACY, results=record)

    return


def evaluate_model(model, tokenizer, epoch, step, dataloaders, device, subsample_size, evaluate_set, epoch_dir):
    '''evaluate means validate or test'''
    # set model to eval mode
    model.eval()
    # calculate number of samples being evaluated
    total_validated_samples = 0
    # calculate total loss and total number of correct labels (weighted acuracy)
    total_loss = 0
    total_accurate = 0
    # turn off grad computation
    with torch.no_grad():
        # evaluate batch by batch
        for batch in dataloaders[evaluate_set]:
            # terminate the process if we are subsampling
            if total_validated_samples > subsample_size:
                break

            # put everything on the right device
            batch =  {k: v.to(device) for k, v in batch.items()}
            batch_size = batch['labels'].shape[0]

            # forward pass in the model
            outputs = model(**batch)

            # accumulate loss and accuracy
            loss, accuracy = customize_loss_and_accuracy(outputs, target=batch['labels'], tokenizer=tokenizer)
            total_loss += loss.item()
            total_accurate += accuracy * batch_size
            total_validated_samples += batch_size


    # calculate the loss and accuracy
    average_loss = total_loss/total_validated_samples
    accuracy = total_accurate/total_validated_samples

    # record the loss and accuracy
    record = {"evaluate_set": evaluate_set, "epoch":epoch, "batch":step,
              "avg_loss": average_loss, 'accuracy': accuracy, 'subsample_size': subsample_size}
    print(record)

    # write to file
    write_results(epoch_dir, result_type=RECORD_TEST_LOSS_AND_ACCURACY, results=record)
    return


def train_epoch(model, tokenizer, epoch, dataloaders, device, optimizer, scheduler, epoch_dir):
    # prepare output dir
    if not os.path.exists(epoch_dir):
        os.makedirs(epoch_dir)

    # evaluate at the beginning of the training
    evaluate_model(model=model, tokenizer=tokenizer, epoch=epoch, step=0, dataloaders=dataloaders, device=device, subsample_size=VAL_SET_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)
    evaluate_model(model=model, tokenizer=tokenizer, epoch=epoch, step=0, dataloaders=dataloaders, device=device, subsample_size=TEST_SET_SIZE, evaluate_set='test', epoch_dir=epoch_dir)

    for step, batch in enumerate(dataloaders['train']):
        # train the batch
        train_batch(model, tokenizer, epoch, step, batch, device, optimizer, scheduler, epoch_dir)

        # validate the model once every 100 steps
        if step % EVAL_PER_STEP == 0:
            evaluate_model(model, tokenizer, epoch, step, dataloaders, device, subsample_size=EVAL_SUBSAMPLE_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)

    # save the model
    write_results(epoch_dir, result_type=RECORD_MODEL, results=model)
    return


def train_model(model, tokenizer, output_dir, dataloaders, optimizer, scheduler, device, hyper_parameters):
    # get device
    model.to(device)

    # number of epochs
    num_train_epochs = hyper_parameters['num_train_epochs']

    # Create the subdirectory for the hyperparameters: this directory is where we will save the result of trainning
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # WRITE hyperparameter to subdirectory
    write_results(output_dir, result_type=RECORD_HYPERPARAMETERS, results=hyper_parameters)


    # train the model on the hyper parameters
    for epoch in range(num_train_epochs):
        epoch_dir = os.path.join(output_dir, f"epoch={epoch}")
        train_epoch(model, tokenizer, epoch, dataloaders, device, optimizer, scheduler, epoch_dir)

    # final evaluation
    evaluate_model(model=model, tokenizer=tokenizer, epoch=num_train_epochs, step="STOP", dataloaders=dataloaders, device=device, subsample_size=VAL_SET_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)
    evaluate_model(model=model, tokenizer=tokenizer, epoch=num_train_epochs, step="STOP", dataloaders=dataloaders, device=device, subsample_size=TEST_SET_SIZE, evaluate_set='test', epoch_dir=epoch_dir)

    return

# T5-small

In [3]:
# load the preprocessed dataset
tokenized_dataset_fp = 'ProcessedDatasets/t5-small/'
tokenized_datasets = load_from_disk(tokenized_dataset_fp)
tokenized_datasets.set_format("torch")
# tokenized_datasets = tokenized_datasets.filter(lambda x: x['enumeration'] == '(9)')
tokenized_datasets = tokenized_datasets.remove_columns(['enumeration'])

# # for testing purposes
# n = 16
# tokenized_datasets['test'] = tokenized_datasets['test'].select(range(n))
# tokenized_datasets['validation'] = tokenized_datasets['validation'].select(range(n))
# tokenized_datasets['train'] = tokenized_datasets['train'].select(range(n))

In [4]:

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

# define hyperparameters
per_device_train_batch_size = 16
learning_rate = 5e-05
num_train_epochs = 5

# define the best hyper parameter
hyper_parameters = {
            'learning_rate': learning_rate,
            'per_device_train_batch_size': per_device_train_batch_size,
            'num_train_epochs': num_train_epochs
        }

# defining the output directory (indicate that it is teacher forcing)
output_dir = f'TrainingData/t5-small/epoch={num_train_epochs}_batch={per_device_train_batch_size}_lr={learning_rate}_teacher/'

# initialize dataloaders
dataloaders = {}
dataloaders['train'] = DataLoader(tokenized_datasets['train'], batch_size=per_device_train_batch_size, shuffle=True)
dataloaders['test'] = DataLoader(tokenized_datasets['test'], batch_size=per_device_train_batch_size)
dataloaders['validation'] = DataLoader(tokenized_datasets['validation'], batch_size=per_device_train_batch_size, shuffle=True)  # shuffle because we want to subsample

# initialize optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# initialize scheduler
# calculate the arguments for shceduler: it depends on the sample size, batch size and epochs
num_training_steps = len(tokenized_datasets['train'])
lr_scheduler_type = 'linear'
lr_scheduler_kwargs = {'optimizer':optimizer,
                        'num_warmup_steps':int(0.1 * num_training_steps),
                        'num_training_steps':int((num_training_steps/per_device_train_batch_size) * num_train_epochs)}
scheduler = get_scheduler(lr_scheduler_type, **lr_scheduler_kwargs)

# define training parameters
training_parameters = {
    'model': model,
    'tokenizer': tokenizer,
    'output_dir': output_dir,
    'dataloaders': dataloaders,
    'optimizer':optimizer,
    'scheduler':scheduler,
    'device':torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'hyper_parameters':hyper_parameters
}

# train the model
train_model(**training_parameters)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24478, 'avg_loss': 0.015598131343722343, 'accuracy': 0.49019607843137253, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24479, 'avg_loss': 0.014387212693691254, 'accuracy': 0.6557377049180327, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24480, 'avg_loss': 0.018121566623449326, 'accuracy': 0.40350877192982454, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24481, 'avg_loss': 0.016472168266773224, 'accuracy': 0.5925925925925926, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24482, 'avg_loss': 0.01609204150736332, 'accuracy': 0.5, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24483, 'avg_loss': 0.01621406152844429, 'accuracy': 0.5172413793103449, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 4, 'batch': 24484, 'avg_loss': 0.015203987248241901, 

In [5]:
from google.colab import runtime
runtime.unassign()